In [1]:
import pymysql
import configparser
import pandas as pd

In [2]:
parser = configparser.ConfigParser()
parser.read('pipeline.conf')

# Credentials for MySQL

hostname = parser.get("mysql_config", "hostname")
port = parser.get("mysql_config", "port")
username = parser.get("mysql_config", "username")
password = parser.get("mysql_config", "password")

In [3]:
conn = pymysql.connect(
    host=hostname,
    user=username,
    password=password,
    port=int(port)
)
if conn is None:
    print("Connection failed")
else:
    print("Connection established!")

Connection established!


In [4]:
cursor = conn.cursor()

In [5]:
cursor.execute("SHOW DATABASES")
cursor.fetchall()

(('board_games',),
 ('datapipe',),
 ('desafio',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('sakila',),
 ('sys',))

In [6]:
cursor.execute("DROP DATABASE IF EXISTS board_games")

1

In [7]:
cursor.execute("CREATE DATABASE board_games")

1

In [8]:
cursor.execute("SHOW DATABASES")
cursor.fetchall()

(('board_games',),
 ('datapipe',),
 ('desafio',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('sakila',),
 ('sys',))

In [9]:
cursor.execute("USE board_games")

0

In [10]:
table_query = ("""

            CREATE TABLE players_data (
                id INT NOT NULL AUTO_INCREMENT,
                jogador_id INT,
                genero VARCHAR(20),
                data_nascimento DATE,
                jogo VARCHAR(50),
                pais VARCHAR(50), 
                num_vitorias INT,
                num_derrotas INT,
                total_partidas INT,
                
                PRIMARY KEY (id)
                )
                """)

cursor.execute(table_query)

#cursor.execute("ALTER TABLE players_data ADD PRIMARY KEY (id)")

0

In [11]:
cursor.execute("DESCRIBE players_data")
cursor.fetchall()

(('id', 'int', 'NO', 'PRI', None, 'auto_increment'),
 ('jogador_id', 'int', 'YES', '', None, ''),
 ('genero', 'varchar(20)', 'YES', '', None, ''),
 ('data_nascimento', 'date', 'YES', '', None, ''),
 ('jogo', 'varchar(50)', 'YES', '', None, ''),
 ('pais', 'varchar(50)', 'YES', '', None, ''),
 ('num_vitorias', 'int', 'YES', '', None, ''),
 ('num_derrotas', 'int', 'YES', '', None, ''),
 ('total_partidas', 'int', 'YES', '', None, ''))

In [12]:
def describe(table):
    query = f"DESCRIBE {table}"
    cursor.execute(query)

    # column names from the description
    columns = [column[0] for column in cursor.description]

    # header row
    header = "+".join(["{:<20}".format(column) for column in columns])
    print(header)

    # separator row
    separator = "+".join(["{:-<20}".format("") for column in columns])
    print(separator)

    # Print the result rows
    results = cursor.fetchall()
    for row in results:
        print("|".join(["{:<20}".format(str(column)) for column in row]))

describe('players_data')


Field               +Type                +Null                +Key                 +Default             +Extra               
--------------------+--------------------+--------------------+--------------------+--------------------+--------------------
id                  |int                 |NO                  |PRI                 |None                |auto_increment      
jogador_id          |int                 |YES                 |                    |None                |                    
genero              |varchar(20)         |YES                 |                    |None                |                    
data_nascimento     |date                |YES                 |                    |None                |                    
jogo                |varchar(50)         |YES                 |                    |None                |                    
pais                |varchar(50)         |YES                 |                    |None                |             

In [13]:
df = pd.read_csv('../data/dados_jogadores.csv', sep=';')
df.head() 

,jogador,genero,data_nascimento,jogo,pais,num_vitorias,num_derrotas,total_partidas
0,jogador 1,Masculino,03/09/1990,Dama,Chile,14.0,0.0,14.0
1,Jogador 2,Feminino,01/05/1986,Jogo da velha,Chile,39.0,20.0,59.0
2,Jogador 3,Masculino,04/10/1964,Jogo da velha,Brasil,26.0,3.0,29.0
3,Jogador 4,Masculino,28/09/1979,Jogo da velha,Argentina,9.0,72.0,81.0
4,Jogador 5,Masculino,03/09/1978,Dama,Uruguai,1.0,54.0,55.0


jogador;genero;data_nascimento;jogo;pais;num_vitorias;num_derrotas;total_partidas
jogador 1;Masculino;03/09/1990;Dama;Chile;14;0;14


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   jogador          499 non-null    object 
 1   genero           499 non-null    object 
 2   data_nascimento  499 non-null    object 
 3   jogo             499 non-null    object 
 4   pais             499 non-null    object 
 5   num_vitorias     491 non-null    float64
 6   num_derrotas     494 non-null    float64
 7   total_partidas   491 non-null    float64
dtypes: float64(3), object(5)
memory usage: 31.3+ KB


In [15]:
df['data_nascimento'] = pd.to_datetime(df['data_nascimento']).dt.strftime('%Y/%m/%d')

/tmp/ipykernel_17080/2786716525.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['data_nascimento'] = pd.to_datetime(df['data_nascimento']).dt.strftime('%Y/%m/%d')


In [16]:
df.head(3)

,jogador,genero,data_nascimento,jogo,pais,num_vitorias,num_derrotas,total_partidas
0,jogador 1,Masculino,1990/03/09,Dama,Chile,14.0,0.0,14.0
1,Jogador 2,Feminino,1986/01/05,Jogo da velha,Chile,39.0,20.0,59.0
2,Jogador 3,Masculino,1964/04/10,Jogo da velha,Brasil,26.0,3.0,29.0


In [17]:
df.isnull().sum()

jogador            0
genero             0
data_nascimento    0
jogo               0
pais               0
num_vitorias       8
num_derrotas       5
total_partidas     8
dtype: int64

In [18]:
df[df.isnull().any(axis=1)]

,jogador,genero,data_nascimento,jogo,pais,num_vitorias,num_derrotas,total_partidas
46,Jogador 47,Masculino,1978/01/31,Dama,Uruguai,NaN,68.0,85.0
55,Jogador 56,Feminino,1959/01/27,Dama,Paraguai,NaN,29.0,80.0
71,Jogador 72,Masculino,2005/03/27,Dama,Paraguai,29.0,7.0,NaN
86,Jogador 87,Feminino,1975/05/25,Dama,Argentina,80.0,6.0,NaN
97,Jogador 98,Feminino,1980/04/16,Dama,Paraguai,NaN,1.0,10.0
137,Jogador 138,Feminino,1963/02/16,Jogo da velha,Chile,NaN,28.0,44.0
246,Jogador 247,Feminino,1954/02/14,Dama,Argentina,33.0,NaN,89.0
251,Jogador 252,Masculino,1985/08/18,Jogo da velha,Chile,NaN,27.0,28.0
255,Jogador 256,Feminino,2000/09/28,Xadrez,Chile,1.0,NaN,73.0
287,Jogador 288,Masculino,1952/01/02,Xadrez,Brasil,3.0,NaN,17.0


In [19]:
df.fillna({'num_vitorias': '-1', 'num_derrotas': '-1', 'total_partidas': '-1'}, inplace=True)

In [20]:
df[df.isnull().any(axis=1)]

,jogador,genero,data_nascimento,jogo,pais,num_vitorias,num_derrotas,total_partidas


In [21]:
df['jogador'] = df['jogador'].str.extract('(\d+)').astype(int)
df.head(3)

,jogador,genero,data_nascimento,jogo,pais,num_vitorias,num_derrotas,total_partidas
0,1,Masculino,1990/03/09,Dama,Chile,14.0,0.0,14.0
1,2,Feminino,1986/01/05,Jogo da velha,Chile,39.0,20.0,59.0
2,3,Masculino,1964/04/10,Jogo da velha,Brasil,26.0,3.0,29.0


                id INT NOT NULL AUTO_INCREMENT,
                jogador_id INT,
                genero VARCHAR(20),
                data_nascimento DATE,
                jogo VARCHAR(50),
                pais VARCHAR(50), 
                num_vitorias INT,
                num_derrotas INT,
                total_partidas INT,

In [22]:
insert_data = ("""
        INSERT INTO players_data
         (
        jogador_id, genero, 
        data_nascimento, jogo, 
        pais, num_vitorias, 
        num_derrotas, total_partidas
         )
          VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
               
               """)
cursor.executemany(insert_data, df.values.tolist())

conn.commit()